In [12]:
import cv2
import numpy as np #going to be used extensively for array operations
import os #helps with filepaths
from matplotlib import pyplot as plt
import time #to provide sleep times between frames being captured
#import mediapipe as mp#for all the keypoints and hand detection going to be used to train our model

In [13]:
actions = np.array(['hello', 'thanks', 'iloveyou'])

#number of videos we want to record for each action
no_sequences = 30 #number of sequences for each action

#number of frames for each video
sequence_length = 30 #number of frames in each sequence

# New Section

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
from zipfile import ZipFile

In [9]:
with ZipFile('drive/MyDrive/Neural_Networks/MP_Data.zip','r') as zipObj:
  zipObj.extractall('drive/MyDrive/Neural_Networks')

In [1]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
label_map = {label:num for num, label in enumerate(actions)}

In [15]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [17]:
DATA_PATH = os.path.join('drive/MyDrive/Neural_Networks/MP_Data')
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
#sequences are eventually our features and labels is our labels itself like hello iloveu and 

In [18]:
np.array(sequences).shape

(90, 30, 1662)

In [19]:
np.array(labels).shape

(90,)

In [20]:
y = to_categorical(labels).astype(int)

In [22]:
x = np.array(sequences)
x.shape

(90, 30, 1662)

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.05)
#train_test_split is a function that helps us in segregating the data
#splitting the training and testing values
#our testing data has been made as 5% of the values(0.05)

In [32]:
y_train.shape

(85, 3)

In [33]:
from tensorflow.keras.models import Sequential #allows us to build a sequential NN
from tensorflow.keras.layers import LSTM, Dense #gives us a temporal components to build our neural network
from tensorflow.keras.callbacks import TensorBoard #if we want to trace and monitor our neural network

In [34]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [59]:
model = Sequential() #we can add layers to our neural network using this
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662))) #64 LSTM units
model.add(LSTM(128, return_sequences=True, activation='relu')) #128 LSTM units
model.add(LSTM(64, return_sequences=False, activation='relu')) #Since the next layer is a dense layer i have set return_sequences=False
model.add(Dense(64, activation='relu')) #64 connected perceptrons
model.add(Dense(32, activation='relu')) #32 
model.add(Dense(actions.shape[0], activation='softmax'))

In [60]:
res = [.7,0.2,0.1]
#why LSTM + MEDIAPIPE over CNN+LSTM
#Less data to produce a hyper accurate model
#Much denser neural network ,way faster to train

In [61]:
actions[np.argmax(res)]

'hello'

In [62]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [63]:
model.fit(x_train, y_train, epochs=500, callbacks=[tb_callback])

Epoch 1/500
3/3 [==============================] - 4s 128ms/step - loss: 1.1253 - categorical_accuracy: 0.3059
Epoch 2/500
3/3 [==============================] - 0s 134ms/step - loss: 1.1128 - categorical_accuracy: 0.3294
Epoch 3/500
3/3 [==============================] - 0s 127ms/step - loss: 1.0390 - categorical_accuracy: 0.6706
Epoch 4/500
3/3 [==============================] - 0s 127ms/step - loss: 1.0040 - categorical_accuracy: 0.5059
Epoch 5/500
3/3 [==============================] - 0s 146ms/step - loss: 0.8319 - categorical_accuracy: 0.6353
Epoch 6/500
3/3 [==============================] - 0s 122ms/step - loss: 0.8068 - categorical_accuracy: 0.6824
Epoch 7/500
3/3 [==============================] - 0s 129ms/step - loss: 0.5915 - categorical_accuracy: 0.6471
Epoch 8/500
3/3 [==============================] - 0s 128ms/step - loss: 3.1315 - categorical_accuracy: 0.6706
Epoch 9/500
3/3 [==============================] - 0s 125ms/step - loss: 0.9501 - categorical_accuracy: 0.5647
E

KeyboardInterrupt: ignored

In [64]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

# Make predictions


In [65]:
res = model.predict(x_test)
res

1/1 [==============================] - 1s 501ms/step


array([[9.9971920e-01, 2.7988417e-04, 1.0052651e-06],
       [1.0572426e-05, 1.9257463e-05, 9.9997020e-01],
       [2.5650161e-08, 2.0012005e-06, 9.9999797e-01],
       [5.2212208e-09, 7.9947517e-07, 9.9999917e-01],
       [5.1239941e-12, 9.9993193e-01, 6.8030422e-05]], dtype=float32)

In [66]:
np.sum(res[0])

1.0000001

In [68]:
for i in range(5):
  print(actions[np.argmax(res[i])])


hello
iloveyou
iloveyou
iloveyou
thanks


In [69]:
for i in range(5):
  print(actions[np.argmax(y_test[i])])


hello
iloveyou
iloveyou
iloveyou
thanks


#Saving our weights

In [83]:
#saving it onto my local machine
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [70]:
model.save('action.h5')

In [71]:
#del model
#model = model_load('action.h5')

NameError: ignored

In [72]:
model.load_weights('action.h5')

# Evaluation using confusion matrix

In [73]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [79]:
yhat = model.predict(x_train)

3/3 [==============================] - 0s 48ms/step


In [80]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [81]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[56,  0],
        [ 0, 29]],

       [[56,  0],
        [ 0, 29]],

       [[58,  0],
        [ 0, 27]]])

In [82]:
accuracy_score(ytrue,yhat)

1.0